<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
    <i>Aprender una técnica no es un fin en si mismo, simplemente indica por donde hay que empezar</i><br><b>Proverbio Japonés</b></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
<i>Material curado y organizado por el Prof. Dr. Hernán D. Merlino</i><br> 
<i>Retrieval Augmented Generation (RAG)  Versión 2.0 - 2024 Q3</i></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>R</strong>etrieval <strong>A</strong>ugmented <strong>G</strong>eneration</p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Cargar librerías</strong></p> 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from transformers import pipeline

<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https://huggingface.co/docs/transformers/main_classes/pipelines" target=\"_blank\">Pipelines Method</a></p>

<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https://huggingface.co/docs/transformers/v4.46.0/en/add_new_pipeline" target=\"_blank\">How to create a custom pipeline?</a></p>

In [4]:
generator = pipeline(task="text-generation",model="datificate/gpt2-small-spanish")

<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https://huggingface.co/models?pipeline_tag=text-generation&sort=trending&search=spanish" target=\"_blank\">Spanish models</a></p>

<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https:///huggingface.co/datificate/gpt2-small-spanish" target=\"_blank\">GPT2-small-spanish is a state-of-the-art language model for Spanish based on the GPT-2 small model</a></p>

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Corpus de documentos</strong></p> 

In [5]:
documents = [
"La inteligencia artificial es una tecnología que permite a las máquinas aprender y hacer tareas inteligentes.",
"El machine learning es una rama de la inteligencia artificial que se enfoca en enseñar a las computadoras mediante datos.",
"Python es un lenguaje popular para el desarrollo de inteligencia artificial debido a sus bibliotecas como TensorFlow y PyTorch.",
"El aprendizaje profundo o deep learning es una técnica avanzada de machine learning que usa redes neuronales profundas.",
"El machine learning es una tecnica de aparendizaje automático."
            ]

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Vectorización</strong></p> 

In [6]:
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Métodos</strong></p> 

In [7]:
def retrieve_document(query, vectorizer, doc_vectors):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, doc_vectors)
    most_similar_idx = similarities.argmax()
    return documents[most_similar_idx]

In [8]:
def rag_example(query):
    document = retrieve_document(query, vectorizer, doc_vectors)
    answer = generator(document,
                       max_length=50,
                       num_return_sequences=1,
                       temperature=0.95,
                       top_k=50,  
                       top_p=0.9)
    return answer[0]["generated_text"]

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong><u>Params</u></strong>:
<ul style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<li><i><b>document:</b></i> texto base</li>
<li><i><b>max_length:</b></i> genera hasta 50 tokens</li>
<li><i><b>num_return_sequences:</b></i> retorna solo un texto</li>
<li><i><b>temperature:</b></i> ajusta la creatividad del modelo</li>
<li><i><b>top_k y top_p:</b></i> controlan la selección de palabras para una generación más coherente y variada</li>
</ul></p>

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Pregunta</strong></p> 

In [11]:
query = "¿Qué es el machine learning?"
answer = rag_example(query)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Respuesta</strong></p> 

In [12]:
print(answer)

El machine learning es una tecnica de aparendizaje automático. Un dispositivo de posicionamiento automático y visualización de información en la mente de un usuario puede ser usado para la orientación de la imagen, las acciones de control de la imagen, la


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 